In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import folium
import warnings
import re
import requests
from IPython.display import display_html 
from bs4 import BeautifulSoup

warnings.filterwarnings("ignore")
pd.options.display.max_seq_items = 2000

# Sources:
# https://www.bikeshare.com/data/
# https://www.bluebikes.com/system-data
# https://crashviewer.nhtsa.dot.gov/CrashAPI
# https://hifld-geoplatform.opendata.arcgis.com/datasets/hospitals/data?selectedAttribute=BEDS Hospital Data
# https://en.wikipedia.org/wiki/List_of_United_States_cities_by_population Cities

In [ ]:
# pd.set_option('display.max_rows', 500)
# pd.set_option('display.max_columns', 500)
# pd.set_option('display.width', 1000)

In [ ]:
# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)
# pd.set_option('display.width', None)
# pd.set_option('display.max_colwidth', -1)

In [ ]:
# Dataset is retrieved from: https://crashviewer.nhtsa.dot.gov/CrashAPI
# GET FARS data from accident, pbtype, and vehicle tables 
fars_acc2018 = pd.read_csv('data/FARS/FARS2018/ACCIDENT.csv')
fars_acc2017 = pd.read_csv('data/FARS/FARS2017/ACCIDENT.csv')
fars_acc2016 = pd.read_csv('data/FARS/FARS2016/ACCIDENT.csv')

fars_PB2018 = pd.read_csv('data/FARS/FARS2018/PBTYPE.csv')
fars_PB2017 = pd.read_csv('data/FARS/FARS2017/PBTYPE.csv')
fars_PB2016 = pd.read_csv('data/FARS/FARS2016/PBTYPE.csv')

fars_veh2018 = pd.read_csv('data/FARS/FARS2018/VEHICLE.csv',encoding= 'unicode_escape')
fars_veh2017 = pd.read_csv('data/FARS/FARS2017/VEHICLE.csv',encoding= 'unicode_escape')
fars_veh2016 = pd.read_csv('data/FARS/FARS2016/VEHICLE.csv',encoding= 'unicode_escape')

In [ ]:
#Join FARS accident and vehicle tables on state and st_case
fars_acc_veh_2018 = pd.merge(fars_acc2018, fars_veh2018,  how='left', left_on=['STATE','ST_CASE'], right_on = ['STATE','ST_CASE'])
fars_acc_veh_2017 = pd.merge(fars_acc2017, fars_veh2017,  how='left', left_on=['STATE','ST_CASE'], right_on = ['STATE','ST_CASE'])
fars_acc_veh_2016 = pd.merge(fars_acc2016, fars_veh2016,  how='left', left_on=['STATE','ST_CASE'], right_on = ['STATE','ST_CASE'])

In [ ]:
#Join FARS accident and vehicle tables on state and st_case
fars_all_2018 = pd.merge(fars_acc_veh_2018, fars_PB2018,  how='left', left_on=['STATE','ST_CASE'], right_on = ['STATE','ST_CASE'])
fars_all_2017 = pd.merge(fars_acc_veh_2017, fars_PB2017,  how='left', left_on=['STATE','ST_CASE'], right_on = ['STATE','ST_CASE'])
fars_all_2016 = pd.merge(fars_acc_veh_2016, fars_PB2016,  how='left', left_on=['STATE','ST_CASE'], right_on = ['STATE','ST_CASE'])

In [ ]:
FARS_16_17_18 = fars_all_2018.append([fars_all_2017, fars_all_2016])

In [ ]:
#GET CRSS data from accident, pbtype, and vehicle tables 
crss_acc2018 = pd.read_csv('data/CRSS/CRSS2018/ACCIDENT.csv')
crss_acc2017 = pd.read_csv('data/CRSS/CRSS2017/ACCIDENT.csv')
crss_acc2016 = pd.read_csv('data/CRSS/CRSS2016/ACCIDENT.csv')

crss_PB2018 = pd.read_csv('data/CRSS/CRSS2018/PBTYPE.csv')
crss_PB2017 = pd.read_csv('data/CRSS/CRSS2017/PBTYPE.csv')
crss_PB2016 = pd.read_csv('data/CRSS/CRSS2016/PBTYPE.csv')

crss_veh2018 = pd.read_csv('data/CRSS/CRSS2018/VEHICLE.csv',encoding= 'unicode_escape')
crss_veh2017 = pd.read_csv('data/CRSS/CRSS2017/VEHICLE.csv',encoding= 'unicode_escape')
crss_veh2016 = pd.read_csv('data/CRSS/CRSS2016/VEHICLE.csv',encoding= 'unicode_escape')

In [ ]:
#Join CRSS accident and vehicle tables on state and st_case
crss_acc_veh_2018 = pd.merge(crss_acc2018, crss_veh2018,  how='left', left_on=['CASENUM'], right_on = ['CASENUM'])
crss_acc_veh_2017 = pd.merge(crss_acc2017, crss_veh2017,  how='left', left_on=['CASENUM'], right_on = ['CASENUM'])
crss_acc_veh_2016 = pd.merge(crss_acc2016, crss_veh2016,  how='left', left_on=['CASENUM'], right_on = ['CASENUM'])

In [ ]:
crss_all_2018 = pd.merge(crss_acc_veh_2018, crss_PB2018,  how='left', left_on=['CASENUM','VEH_NO'], right_on = ['CASENUM','VEH_NO'])
crss_all_2017 = pd.merge(crss_acc_veh_2017, crss_PB2017,  how='left', left_on=['CASENUM','VEH_NO'], right_on = ['CASENUM','VEH_NO'])
crss_all_2016 = pd.merge(crss_acc_veh_2016, crss_PB2016,  how='left', left_on=['CASENUM','VEH_NO'], right_on = ['CASENUM','VEH_NO'])

In [ ]:
CRSS_16_17_18 = crss_all_2018.append([crss_all_2017, crss_all_2016])

In [ ]:
# Web Scrap to collect biggest US cities by population
# Get URL, request HTML and create soup
URL = 'https://en.wikipedia.org/wiki/List_of_United_States_cities_by_population'
page = requests.get(URL)
page.content
soup = BeautifulSoup(page.content, 'html.parser')

# Find all tables and table for interest is table 4, extract it and send it to DF
table = soup.find_all('table')
top_cities = pd.read_html(str(table))[4]

# cities_df = pd.read_csv('data/Cities.csv')
#Extract Lat and Long independently to plot cities and convert to float
top_cities['Lat'] = top_cities['Location'].str.extract('(\d+\.\d+)').astype(float)
top_cities['Lon'] = (top_cities['Location'].str.extract('\s(\d+\.\d+)').astype(float))*-1

#Convert Lat and Long to radians to faciliate creating a radius for the city
top_cities['Lat_rad'] = top_cities['Lat'] * np.pi / 180
top_cities['Lon_rad'] = top_cities['Lon'] * np.pi / 180

# Get land area in km2 (Clean and extract number only)
top_cities['Land'] = top_cities['2016 land area.1'].str.extract('([\d,]+\.\d+)').replace(',','')
top_cities['Land'] = top_cities['Land'].str.replace(',','').astype(float)

#Clean City name
top_cities['City'] = top_cities['City'].str.replace(r"\[.*\]", '')
# top_cities

# #Remove commas and units for numerical values
# cities_df['Estimate 2019'] = cities_df['Estimate 2019'].str.replace(',','').astype(float)
# cities_df['Census 2010'] = cities_df['Census 2010'].str.replace(',','').astype(float)

In [ ]:
##PLOT a count of accidents by year and month
yr_month_count=FARS_16_17_18.groupby(['YEAR','MONTH_x']).ST_CASE.nunique().reset_index().rename(columns={'ST_CASE':'count'})
Months_name = {1:"Jan", 2:"Feb", 3:"Mar", 4:"Apr", 5:"May", 6:"Jun", 
          7:"Jul", 8:"Aug", 9:"Sep", 10:"Oct", 11:"Nov", 12:"Dec"}
yr_month_count=yr_month_count.replace({"MONTH_x": Months_name})


months = ["Jan", "Feb", "Mar", "Apr", "May", "Jun", 
          "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]
yr_month_count['month'] = pd.Categorical(yr_month_count['MONTH_x'], categories=months, ordered=True)

In [ ]:
#Comparison of number of fatal accidents from 2016 to 2018
months = ["Jan", "Feb", "Mar", "Apr", "May", "Jun", 
          "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]
yr_month_count['month'] = pd.Categorical(yr_month_count['MONTH_x'], categories=months, ordered=True)
plt.figure(figsize=(20,9))
ax =sns.lineplot(data = yr_month_count, x='month',y='count', hue='YEAR',style="YEAR",ci=None,palette="Paired",markers=True)
ax.set_title('Comparing total number of accidents per month, 2016 To 2018',fontsize=20)
sns.set(font_scale=2)
sns.set_style("white")

plt.show()

In [ ]:
pb=FARS_16_17_18[FARS_16_17_18['PBPTYPE'].notna()]
yr_month_count_pb=pb.groupby(['YEAR','MONTH_x']).ST_CASE.nunique().reset_index().rename(columns={'ST_CASE':'count'})

Months_name = {1:"Jan", 2:"Feb", 3:"Mar", 4:"Apr", 5:"May", 6:"Jun", 
          7:"Jul", 8:"Aug", 9:"Sep", 10:"Oct", 11:"Nov", 12:"Dec"}
yr_month_count_pb=yr_month_count_pb.replace({"MONTH_x": Months_name})

months = ["Jan", "Feb", "Mar", "Apr", "May", "Jun", 
          "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]
yr_month_count_pb['month'] = pd.Categorical(yr_month_count_pb['MONTH_x'], categories=months, ordered=True)
plt.figure(figsize=(20,9))
ax =sns.lineplot(data = yr_month_count_pb, x='month',y='count', hue='YEAR',style="YEAR",ci=None,palette="Paired",markers=True)
ax.set_title('Comparing total number of accidents involves Pedestrian/Bikes per month, 2016 To 2018',fontsize=20)
sns.set(font_scale=2)
sns.set_style("white")

plt.show()

In [ ]:
#Create copy of data
FARS=FARS_16_17_18.copy()
CRSS=CRSS_16_17_18.copy()
#Replace integer with actual values
Light_Cond = {1:"Daylight", 2:"Dark – Not Lighted", 3:"Dark – Lighted", 4:"Dawn", 5:"Dusk", 6:"Dark – Unknown Lighting", 
          7:"Other", 8:"Not Reported", 9:"Unknown/Reported as Unknown", 10:"Oct", 11:"Nov", 12:"Dec"}
FARS=FARS.replace({"LGT_COND": Light_Cond})
CRSS=CRSS.replace({"LGT_COND": Light_Cond})

Alc_inv = {1:"Alcohol Involved", 2:"No Alcohol Involved",  8:"No Applicable Person",9:"Unknown"}
CRSS=CRSS.replace({"ALCOHOL": Alc_inv})

Weather_str={0:"No Additional Atmospheric Conditions", 1:"Clear", 2:"Rain", 3:"Sleet or Hail", 4:"Snow", 
         5:"Fog, Smog, Smoke", 6:"Severe Crosswinds", 
        7:"Blowing Sand, Soil, Dirt", 8:"Other",  
        10:"Cloudy", 11:"Blowing Snow", 12:"Freezing Rain or Drizzle",
        98:"Unknown/Reported as Unknown",
        99:"Unknown/Reported as Unknown"}
FARS=FARS.replace({"WEATHER": Weather_str})
CRSS=CRSS.replace({"WEATHER": Weather_str})

Manner_collison={0:"Not Collision with Motor Vehicle in Transport", 1:"Front-to-Rear", 2:"Front-to-Front",
                 3:"Angle – Front-to-Side, Same Direction", 4:"Angle – Front-to-Side, Opposite Direction", 
         5:"Angle – Front-to-Side, Right Angle (Includes Broadside)", 
        6:"Angle – Front-to-Side/Angle-Direction Not Specified", 
        7:"Sideswipe – Same Direction", 8:"Sideswipe – Opposite Direction",
        9:"Rear-to-Side",
        10:"Rear-to-Rear", 11:"Other (End-Swipes and Others)", 
        98:"Not Reported",
        99:"Unknown/Reported as Unknown"}
FARS=FARS.replace({"MAN_COLL_x": Manner_collison})
CRSS=CRSS.replace({"MAN_COLL_x": Manner_collison})


Rel_Roads={1:"On Roadway", 2:"On Shoulder", 3:"On Median", 4:"On Roadside", 
         5:"Outside Trafficway", 6:"Off Roadway – Location Unknown", 
        7:"In Parking Lane/Zone", 8:"Gore",  
        10:"Separator", 11:"Continuous Left Turn Lane", 
        12:"Unknown/Reported as Unknown",
        98:"Not Reported",
        99:"Unknown/Reported as Unknown"}

FARS=FARS.replace({"REL_ROAD": Rel_Roads})
CRSS=CRSS.replace({"REL_ROAD": Rel_Roads})


Level_Damage_Veh={0:"No Damage", 2:"Minor Damage", 4:"Functional Damage", 
         6:"Disabling Damage", 8:"Not Reported", 
     9:"Reported as Unknown"
        }

FARS=FARS.replace({"DEFORMED": Level_Damage_Veh})
CRSS=CRSS.replace({"DEFORMED": Level_Damage_Veh})


Haz_Involve={1:"No", 2:"Yes"
        }

FARS=FARS.replace({"HAZ_INV": Haz_Involve})
CRSS=CRSS.replace({"HAZ_INV": Haz_Involve})


### Let's compare major factors between fatal crash and injury-only crash.

##### First, let's take a look at light condition between Fatal and Injury-only Crash.
From the visualization, the majority of accidents in both types of accidents happen under the condition of 'Daylight'. However, more percentage of fatal accidents happened in 'Dark-Lighted', 'Dark-Not lighted' compared to injury-only accidents. 
So light condition is an important factor.

In [ ]:
FARS.dropna(subset=['LGT_COND','VE_TOTAL','DRUNK_DR'],inplace = True)
CRSS.dropna(subset=['LGT_COND','VE_TOTAL','ALCOHOL'],inplace = True)
FARS_sort = FARS.sort_values('LGT_COND',ascending=True)
CRSS_sort = CRSS.sort_values('LGT_COND',ascending=True)
f,ax=plt.subplots(1,2,figsize=(15,10))
sns.set(font_scale=1)
rc={'font.size': 10, 'axes.labelsize': 12, 
    'axes.titlesize': 15, 'xtick.labelsize':8, 'ytick.labelsize': 8}
sns.set_context(rc=rc)
sns.set_style("white")

sns.histplot(FARS_sort["LGT_COND"],ax=ax[0])
ax[0].set_title('Distribution of Light Condition - Fatal Accident')
plt.setp(ax[0].get_xticklabels(), rotation=90)



sns.histplot(CRSS_sort["LGT_COND"],ax=ax[1])
ax[1].set_title('Distribution of Light Condition - Injury-Only')
plt.setp(ax[1].get_xticklabels(), rotation=90)
plt.show()


##### Next, let's take a look and see if single car accident/multi-car accident is another factor leading to fatal accidents.
So from the chart below, it is surprisingly that among fatal accidents, 36% of fatal accidents are single vehicle accidents while only 16% of injury-only are single-vehicle accidents. Both fatal and injury-only accidents has 2-vehicle accidents as the majority. 

In [ ]:
#from IPython.display import display_html 
FARS_ve=FARS.groupby(['VE_TOTAL'])['ST_CASE'].count().reset_index().rename(columns={'ST_CASE':'COUNT'})
FARS_ve=FARS_ve.set_index(['VE_TOTAL'])
FARS_ve["%"] = FARS_ve.apply(lambda x:  100*x / x.sum())

CRSS_ve=CRSS.groupby(['VE_TOTAL'])['CASENUM'].count().reset_index().rename(columns={'ST_CASE':'COUNT'})
CRSS_ve=CRSS_ve.set_index(['VE_TOTAL'])
CRSS_ve["%"] = CRSS_ve.apply(lambda x:  100*x / x.sum())

FARS_ve_styler = FARS_ve.style.set_table_attributes("style='display:inline'").set_caption('Fatal-accident')
CRSS_ve_styler = CRSS_ve.style.set_table_attributes("style='display:inline'").set_caption('Injury-Only accident')

display_html(FARS_ve_styler._repr_html_()+CRSS_ve_styler._repr_html_(), raw=True)

In [ ]:
f,ax=plt.subplots(1,2,figsize=(15,10))
sns.set(font_scale=1)
rc={'font.size': 10, 'axes.labelsize': 12, 
    'axes.titlesize': 15, 'xtick.labelsize':10, 'ytick.labelsize': 10}
sns.set_context(rc=rc)
sns.set_style("white")
sns.histplot(FARS["VE_TOTAL"],ax=ax[0],binwidth=1)
ax[0].set_title('Distribution of Total Vehicles Involved - Fatal Accident')
ax[0].set(xlim=(0,8))

sns.histplot(CRSS["VE_TOTAL"],ax=ax[1],binwidth=1)
ax[1].set_title('Distribution of Total Vehicles Involved - Injury-Only')
ax[1].set(xlim=(0,8))
    
plt.show()



##### As alcohol involvement is a critical factor for accident, let's compare it.
It's good that we see the majority in both fatal and injury-only accidents are non-alcohol involved.
However, if we compare fatal with injury-only, we could see that 24.73% of fatal accidents has at least 1 drunk driver involved.

In [ ]:
#from IPython.display import display_html 
FARS_al=FARS.groupby(['DRUNK_DR'])['ST_CASE'].count().reset_index().rename(columns={'ST_CASE':'COUNT'})
FARS_al=FARS_al.set_index(['DRUNK_DR'])
FARS_al["%"] = FARS_al.apply(lambda x:  100*x / x.sum())

CRSS_al=CRSS.groupby(['ALCOHOL'])['CASENUM'].count().reset_index().rename(columns={'ST_CASE':'COUNT'})
CRSS_al=CRSS_al.set_index(['ALCOHOL'])
CRSS_al["%"] = CRSS_al.apply(lambda x:  100*x / x.sum())

FARS_al_styler = FARS_al.style.set_table_attributes("style='display:inline'").set_caption('Fatal-accident')
CRSS_al_styler = CRSS_al.style.set_table_attributes("style='display:inline'").set_caption('Injury-Only accident')

display_html(FARS_al_styler._repr_html_()+CRSS_al_styler._repr_html_(), raw=True)

In [ ]:
f,ax=plt.subplots(1,2,figsize=(15,10))
sns.set(font_scale=1)
rc={'font.size': 10, 'axes.labelsize': 12, 
    'axes.titlesize': 15, 'xtick.labelsize':10, 'ytick.labelsize': 10}
sns.set_context(rc=rc)
sns.set_style("white")
sns.histplot(FARS["DRUNK_DR"],ax=ax[0],binwidth=1)
ax[0].set_title('Distribution of Drunk Driver Involved - Fatal Accident')
#ax[0].set(xlim=(0,5))

sns.histplot(CRSS["ALCOHOL"],ax=ax[1],binwidth=2)
ax[1].set_title('Distribution of Alcohol Involved - Injury-Only')
plt.setp(ax[1].get_xticklabels(), rotation=90)


plt.show()

##### Now, let's see if weather will lead to more fatal accidents.
From the chart, we could see that the weather distribution is very identical between two type of accidents.
Thus, we may know that weather isn't a key factor that cause fatal accident.

In [ ]:
FARS_w=FARS.groupby(['WEATHER'])['ST_CASE'].count().reset_index().rename(columns={'ST_CASE':'COUNT'})
FARS_w=FARS_w.set_index(['WEATHER'])
FARS_w["%"] = FARS_w.apply(lambda x:  100*x / x.sum())

CRSS_w=CRSS.groupby(['WEATHER'])['CASENUM'].count().reset_index().rename(columns={'ST_CASE':'COUNT'})
CRSS_w=CRSS_w.set_index(['WEATHER'])
CRSS_w["%"] = CRSS_w.apply(lambda x:  100*x / x.sum())

FARS_w_styler = FARS_w.style.set_table_attributes("style='display:inline'").set_caption('Fatal-accident')
CRSS_w_styler = CRSS_w.style.set_table_attributes("style='display:inline'").set_caption('Injury-Only accident')

display_html(FARS_w_styler._repr_html_()+CRSS_w_styler._repr_html_(), raw=True)

In [ ]:
FARS_sort = FARS.sort_values('WEATHER',ascending=True)
CRSS_sort = CRSS.sort_values('WEATHER',ascending=True)

f,ax=plt.subplots(1,2,figsize=(15,10))
sns.set(font_scale=1)
rc={'font.size': 10, 'axes.labelsize': 12, 
    'axes.titlesize': 15, 'xtick.labelsize':8, 'ytick.labelsize': 8}
sns.set_context(rc=rc)
sns.set_style("white")
#FARS.dropna(subset=['WEATHER'],inplace = True)
#CRSS.dropna(subset=['WEATHER'],inplace = True)

sns.histplot(FARS_sort["WEATHER"],ax=ax[0])
ax[0].set_title('Distribution of Weather - Fatal Accident')
plt.setp(ax[0].get_xticklabels(), rotation=90)

sns.histplot(CRSS_sort["WEATHER"],ax=ax[1])
ax[1].set_title('Distribution of Weather - Injury-Only')
plt.setp(ax[1].get_xticklabels(), rotation=90)
plt.show()

##### Next, we will take a look at how the collion manner different between both accidents.
What we have noticed that 43.3% of fatal accidents are 'Not Collision with Motor Vehicle in Transport' which aligns with what we have seen in the factor or single-vehicle/muti-vehicle.
Besides, we find out that more (14.36%) fatal accidents has front-to-front manner of collision compared to 3.7% injury-only accident. 


In [ ]:
FARS_col=FARS.groupby(['MAN_COLL_x'])['ST_CASE'].count().reset_index().rename(columns={'ST_CASE':'COUNT'})
FARS_col=FARS_col.set_index(['MAN_COLL_x'])
FARS_col["%"] = FARS_col.apply(lambda x:  100*x / x.sum())

CRSS_col=CRSS.groupby(['MAN_COLL_x'])['CASENUM'].count().reset_index().rename(columns={'ST_CASE':'COUNT'})
CRSS_col=CRSS_col.set_index(['MAN_COLL_x'])
CRSS_col["%"] = CRSS_col.apply(lambda x:  100*x / x.sum())

FARS_col_styler = FARS_col.style.set_table_attributes("style='display:inline'").set_caption('Fatal-accident')
CRSS_col_styler = CRSS_col.style.set_table_attributes("style='display:inline'").set_caption('Injury-Only accident')

display_html(FARS_col_styler._repr_html_()+CRSS_col_styler._repr_html_(), raw=True)

In [ ]:
FARS_sort = FARS.sort_values('MAN_COLL_x',ascending=True)
CRSS_sort = CRSS.sort_values('MAN_COLL_x',ascending=True)

f,ax=plt.subplots(1,2,figsize=(15,10))
sns.set(font_scale=1)
rc={'font.size': 10, 'axes.labelsize': 12, 
    'axes.titlesize': 15, 'xtick.labelsize':8, 'ytick.labelsize': 8}
sns.set_context(rc=rc)
sns.set_style("white")
#FARS.dropna(subset=['WEATHER'],inplace = True)
#CRSS.dropna(subset=['WEATHER'],inplace = True)

sns.histplot(FARS_sort["MAN_COLL_x"],ax=ax[0])
ax[0].set_title('Distribution of Manner of Collision - Fatal Accident')
plt.setp(ax[0].get_xticklabels(), rotation=90)

sns.histplot(CRSS_sort["MAN_COLL_x"],ax=ax[1])
ax[1].set_title('Distribution of Manner of Collision - Injury-Only')
plt.setp(ax[1].get_xticklabels(), rotation=90)
plt.show()

##### We are curious to see the position of the crash relates to the trafficway.
The Rel_road variable identifies the location of the crash as it relates to its position within or outside the trafficway.
Combine this information with the manner of collision, we could get more sense of how and where the collison happened in a traffic way.

When we compare both types of accidents, the majority happens 'On Roadway'.
What differs is more (19.81%) fatal accidents happen 'On Roadside' compared to 7.41% of injury-only accident. This further validates that single-vehicle or vehicle that is not in collision with another vehicle is more common in fatal accident.

In [ ]:
FARS_rel=FARS.groupby(['REL_ROAD'])['ST_CASE'].count().reset_index().rename(columns={'ST_CASE':'COUNT'})
FARS_rel=FARS_rel.set_index(['REL_ROAD'])
FARS_rel["%"] = FARS_rel.apply(lambda x:  100*x / x.sum())

CRSS_rel=CRSS.groupby(['REL_ROAD'])['CASENUM'].count().reset_index().rename(columns={'ST_CASE':'COUNT'})
CRSS_rel=CRSS_rel.set_index(['REL_ROAD'])
CRSS_rel["%"] = CRSS_rel.apply(lambda x:  100*x / x.sum())

FARS_rel_styler = FARS_rel.style.set_table_attributes("style='display:inline'").set_caption('Fatal-accident')
CRSS_rel_styler = CRSS_rel.style.set_table_attributes("style='display:inline'").set_caption('Injury-Only accident')

display_html(FARS_rel_styler._repr_html_()+CRSS_rel_styler._repr_html_(), raw=True)

In [ ]:
FARS.dropna(subset=['REL_ROAD'],inplace = True)
CRSS.dropna(subset=['REL_ROAD'],inplace = True)
FARS_sort = FARS.sort_values('REL_ROAD',ascending=True)
CRSS_sort = CRSS.sort_values('REL_ROAD',ascending=True)

f,ax=plt.subplots(1,2,figsize=(15,10))
sns.set(font_scale=1)
rc={'font.size': 10, 'axes.labelsize': 12, 
    'axes.titlesize': 15, 'xtick.labelsize':8, 'ytick.labelsize': 8}
sns.set_context(rc=rc)
sns.set_style("white")
#FARS.dropna(subset=['WEATHER'],inplace = True)
#CRSS.dropna(subset=['WEATHER'],inplace = True)

sns.histplot(FARS_sort["REL_ROAD"],ax=ax[0])
ax[0].set_title('Distribution of Relationship to TrafficWay - Fatal Accident')
plt.setp(ax[0].get_xticklabels(), rotation=90)

sns.histplot(CRSS_sort["REL_ROAD"],ax=ax[1])
ax[1].set_title('Distribution of Relationship to TrafficWay - Injury-Only')
plt.setp(ax[1].get_xticklabels(), rotation=90)
plt.show()

##### According to WHO, accidents that involve hazardous material may lead to more fatal accidents.
Our dataset sample doesn't include much data records which has hazardous material involvement. But by quick check, we still see difference between fatal accidents and injury-only accident. Although we can't conclude that this factor plays large part, but 0.28% of our fatal-accidents records is hazardous material involved, while only 0.05% of our injury-only accidents are hazardous material involved.

In [ ]:
FARS_haz=FARS.groupby(['HAZ_INV'])['ST_CASE'].count().reset_index().rename(columns={'ST_CASE':'COUNT'})
FARS_haz=FARS_haz.set_index(['HAZ_INV'])
FARS_haz["%"] = FARS_haz.apply(lambda x:  100*x / x.sum())

CRSS_haz=CRSS.groupby(['HAZ_INV'])['CASENUM'].count().reset_index().rename(columns={'ST_CASE':'COUNT'})
CRSS_haz=CRSS_haz.set_index(['HAZ_INV'])
CRSS_haz["%"] = CRSS_haz.apply(lambda x:  100*x / x.sum())

FARS_haz_styler = FARS_haz.style.set_table_attributes("style='display:inline'").set_caption('Fatal-accident')
CRSS_haz_styler = CRSS_haz.style.set_table_attributes("style='display:inline'").set_caption('Injury-Only accident')

display_html(FARS_haz_styler._repr_html_()+CRSS_haz_styler._repr_html_(), raw=True)

In [ ]:
#FARS.dropna(subset=['HAZ_INV'],inplace = True)
#CRSS.dropna(subset=['HAZ_INV'],inplace = True)

f,ax=plt.subplots(1,2,figsize=(15,10))
sns.set(font_scale=1)
rc={'font.size': 10, 'axes.labelsize': 12, 
    'axes.titlesize': 15, 'xtick.labelsize':8, 'ytick.labelsize': 8}
sns.set_context(rc=rc)
sns.set_style("white")
#FARS.dropna(subset=['WEATHER'],inplace = True)
#CRSS.dropna(subset=['WEATHER'],inplace = True)

sns.histplot(FARS["HAZ_INV"],ax=ax[0])
ax[0].set_title('Distribution of Hazordous Material Involvement - Fatal Accident')
plt.setp(ax[0].get_xticklabels(), rotation=90)

sns.histplot(CRSS["HAZ_INV"],ax=ax[1])
ax[1].set_title('Distribution of Hazordous Material Involvement - Injury-Only')
plt.setp(ax[1].get_xticklabels(), rotation=90)
plt.show()

In [ ]:
test_bar=FARS.groupby(['DEFORMED','HAZ_INV'])['ST_CASE'].count().reset_index().rename(columns={'ST_CASE':'COUNT'})
test_bar1=CRSS.groupby(['DEFORMED','HAZ_INV'])['CASENUM'].count().reset_index().rename(columns={'CASENUM':'COUNT'})

In [ ]:
#test_bar1=test_bar1.set_index(['DEFORMED',"HAZ_INV"])
#test_bar1["%"] = test_bar1.groupby(level=0).apply(lambda x:  100*x / x.sum())
#test_bar1

In [ ]:
#test_bar=test_bar.set_index(['DEFORMED',"HAZ_INV"])
#test_bar["%"] = test_bar.groupby(level=0).apply(lambda x:  100*x / x.sum())
#test_bar

In [ ]:
#sns.barplot(x = 'DEFORMED',
 #           y = 'COUNT',
  #          hue='HAZ_INV',
   #         data = test_bar)

### Now, we will focus more on fatal-crash and find if there's any patterns among this type of accident.

In [ ]:
heatmap_dam_ve=FARS.groupby(['DEFORMED','VE_TOTAL'])['ST_CASE'].count().reset_index().rename(columns={'ST_CASE':'COUNT'})
#test_heatmap

In [ ]:
fig, ax = plt.subplots(figsize=(20,15))         # Sample figsize in inches

heatmap_dam_ve_2 = pd.pivot_table(data=heatmap_dam_ve,
                    index='DEFORMED',
                    values='COUNT',
                    columns='VE_TOTAL')
sns.heatmap(heatmap_dam_ve_2,cmap='coolwarm',annot=True,fmt=".1f",annot_kws={'size':12},linewidths=.5, ax=ax)
plt.show()

##### Let's see if speeding is directly related to fatal accident.
Speed_rel records whether the driver's speed was related to the crash as indicated by law enforcement.

So we can see that 21.31% of fatal accidents is directly related to speeding. Among which, 6.9% exceeded speed limit and 7.6% is too fast for conditions. This implies that speeding is one of the major reason leads to fatal accident. Although we have educated drivers not to exceed speed limit and drive not too fast, there are still lots of space to improve.

In [ ]:
FARS_sp=FARS.groupby(['SPEEDREL'])['ST_CASE'].count().reset_index().rename(columns={'ST_CASE':'COUNT'})
FARS_sp=FARS_sp.set_index(['SPEEDREL'])
FARS_sp["%"] = FARS_sp.apply(lambda x:  100*x / x.sum())

CRSS_sp=CRSS.groupby(['SPEEDREL'])['CASENUM'].count().reset_index().rename(columns={'ST_CASE':'COUNT'})
CRSS_sp=CRSS_sp.set_index(['SPEEDREL'])
CRSS_sp["%"] = CRSS_sp.apply(lambda x:  100*x / x.sum())

FARS_sp_styler = FARS_sp.style.set_table_attributes("style='display:inline'").set_caption('Fatal-accident')
CRSS_sp_styler = CRSS_sp.style.set_table_attributes("style='display:inline'").set_caption('Injury-Only accident')

display_html(FARS_sp_styler._repr_html_()+CRSS_sp_styler._repr_html_(), raw=True)

In [ ]:
Speed_rel={0:"No", 2:"Yes, Racing", 3:"Yes, Exceeded Speed Limit", 
         4:"Yes, Too Fast for Conditions", 5:"Yes, Specifics Unknown",
           8:"No Driver Present/Unknown if Driver Present",
     9:"Reported as Unknown"
        }

FARS=FARS.replace({"SPEEDREL": Speed_rel})



conditions = [
    (FARS['SPEEDREL'] == 'Yes, Racing') | (FARS['SPEEDREL'] == 'Yes, Exceeded Speed Limit')|(FARS['SPEEDREL'] == 'Yes, Too Fast for Conditions')
    |(FARS['SPEEDREL'] == 'Yes, Specifics Unknown')
    ,(FARS['SPEEDREL'] == 'No')| (FARS['SPEEDREL'] == 'No Driver Present/Unknown if Driver Present'),
 (FARS['SPEEDREL'] == 'Reported as Unknown')]
choices = ['Yes, speed related crash', 'No','Unknown']
FARS['SPEEDREL_BIN'] = np.select(conditions, choices, default='Unknown')

##### We already know that speed is critical factor, and let's combine it with the level of damage of vehicles.
From the box plot, we can confirm that the level of damage is more severe as travelling speed increases.
We have further categorize the accident based on speed_rel and the color grey marks the accident which is speed related according to law enforcement. 
We could see that in the most severe damage 'Disabling damage', the overall travelling speed is higher compared to the other level of damage. And at the same time, the 'speed related crash' has the highest median speed at 69 mph.
From this plot, we could learn that speed is also directly related to vehicle level of damage.

In [ ]:
FARS_Trv_sp = FARS[FARS['TRAV_SP']<997]
FARS_Trv_sp = FARS_Trv_sp.sort_values('SPEEDREL_BIN',ascending=True)
plt.figure(figsize=(15,8))
ax=sns.boxplot(x = 'DEFORMED', y = 'TRAV_SP', hue="SPEEDREL_BIN", data = FARS_Trv_sp,linewidth=2,fliersize=5,palette="Set2") 
_ = ax.set_xticklabels(ax.get_xticklabels(), rotation=0)
sns.set(font_scale =1)
ax.set(ylim=(0, 200))
ax.set_title('Travel Speed by Damage Level',fontsize=18)
sns.set_style("white")

#Let's show the median values in the plot.
m1 = FARS_Trv_sp.groupby(['DEFORMED',"SPEEDREL_BIN"])['TRAV_SP'].median().values
mL1 = [str(np.round(s, 2)) for s in m1]


ind = 0
for tick in range(len(ax.get_xticklabels())):
    ax.text(tick, m1[ind+1]+m1[ind+1]*0.1, mL1[ind+1],  horizontalalignment='center', size='medium', color='blue', weight='bold')
    ax.text(tick+0.25, m1[ind+2]+m1[ind+2]*0.1, mL1[ind+2],  horizontalalignment='center', size='medium', color='blue', weight='bold')
    ax.text(tick-0.25, m1[ind]+m1[ind]*0.1, mL1[ind], horizontalalignment='center', size='medium',color='blue', weight='bold')
    ind += 3 

plt.show()


In [ ]:
FARS_Trv_sp.groupby(['DEFORMED',"SPEEDREL_BIN"])['TRAV_SP'].median()

In [ ]:
#Rur_Urban={1:"Rural", 2:"Urban",  6:"Trafficway Not in State Inventory", 
 #        8:"Not Reported",9:"Unknown"}

#FARS=FARS.replace({"RUR_URB": Rur_Urban})

#plt.figure(figsize=(15,8))
#ax=sns.boxplot(x = 'RUR_URB', y = 'FATALS', data = FARS,linewidth=2,fliersize=5) 
#_# = ax.set_xticklabels(ax.get_xticklabels(), rotation=90)
#sns.set(font_scale =1)
#ax.set(ylim=(0, 10))
#ax.set_title('Fatals by Rural/Urban',fontsize=18)
#sns.set_style("white")

#Let's show the median values in the plot.
#mean3 = FARS.groupby(['RUR_URB'])['FATALS'].mean()
#mean=mean3.round(decimals=2)
#vertical_offset = FARS['FATALS'].mean() * 0.5 # offset from median for display

#for xtick in ax.get_xticks():
 #   ax.text(xtick,mean[xtick] + vertical_offset,mean[xtick], 
 #           horizontalalignment='center',size='medium',color='blue',weight='semibold')

#plt.show()

In [ ]:
#route_str={1:"Interstate", 2:"U.S. Highway", 3:"State Highway",4:"County Road", 
#       5:"Local Street – Township",6:"Local Street – Municipality", 
#       7:"Local Street – Frontage Road (Since 1994)",
#         8:"Other",9:"Unknown"}

#FARS=FARS.replace({"ROUTE": route_str})

#plt.figure(figsize=(15,8))
#ax=sns.boxplot(x = 'ROUTE', y = 'FATALS', data = FARS,linewidth=2,fliersize=5) 
#_ = ax.set_xticklabels(ax.get_xticklabels(), rotation=90)
#sns.set(font_scale =1)
#ax.set(ylim=(0, 10))
#ax.set_title('Fatals by Rural/Urban',fontsize=18)
#sns.set_style("white")

#Let's show the median values in the plot.
#mean2 = FARS.groupby(['ROUTE'])['FATALS'].mean()
#mean=mean2.round(decimals=2)
#vertical_offset = FARS['FATALS'].mean() * 0.5 # offset from median for display

#for xtick in ax.get_xticks():
#    ax.text(xtick,mean[xtick] + vertical_offset,mean[xtick], 
#            horizontalalignment='center',size='medium',color='blue',weight='semibold')

#plt.show()

In [ ]:
FARS.columns

In [ ]:
CRSS.columns

In [ ]:
#Filtering records containing Not, Arr and Hosp Time
FARS_hosp_time = FARS_16_17_18[(FARS_16_17_18['NOT_HOUR']!=99) & (FARS_16_17_18['NOT_HOUR']!=88) & 
                                      (FARS_16_17_18['ARR_HOUR']!=99) & (FARS_16_17_18['ARR_HOUR']!=88) & 
                                      (FARS_16_17_18['HOSP_HR']!=99) & (FARS_16_17_18['HOSP_HR']!=88)&
                              (FARS_16_17_18['NOT_MIN']!=99)&(FARS_16_17_18['NOT_MIN']!=88)&
                              (FARS_16_17_18['ARR_MIN']!=99)&(FARS_16_17_18['ARR_MIN']!=88)&
                              (FARS_16_17_18['HOSP_MN']!=99)&(FARS_16_17_18['HOSP_MN']!=88)]
FARS_hosp_time1 = FARS_hosp_time[['NOT_HOUR', 'NOT_MIN', 'ARR_HOUR', 'ARR_MIN', 'HOSP_HR', 'HOSP_MN']]
#FARS_hosp_time1['NOT_MIN'].unique()

In [ ]:
FARS_hosp_time['Notification_Time'] = (pd.to_datetime(FARS_hosp_time['YEAR'].astype(str) + ':'+
                                                      FARS_hosp_time['MONTH_x'].astype(str) + ':'+
                                                      FARS_hosp_time['DAY_x'].astype(str) + ':'+
                                                      FARS_hosp_time['NOT_HOUR'].astype(str) + ':' +
                                                            FARS_hosp_time['NOT_MIN'].astype(str), 
                                                            format='%Y:%m:%d:%H:%M'))


In [ ]:
FARS_hosp_time['Arrival_Time'] = (pd.to_datetime(FARS_hosp_time['YEAR'].astype(str) + ':'+
                                                 FARS_hosp_time['MONTH_x'].astype(str) + ':'+
                                                 FARS_hosp_time['DAY_x'].astype(str) + ':'+
                                                 FARS_hosp_time['ARR_HOUR'].astype(str) + ':' +
                                                 FARS_hosp_time['ARR_MIN'].astype(str), 
                                                             format='%Y:%m:%d:%H:%M'))

In [ ]:
FARS_hosp_time['Hospital_Time'] = (pd.to_datetime(FARS_hosp_time['YEAR'].astype(str) + ':'+
                                                      FARS_hosp_time['MONTH_x'].astype(str) + ':'+
                                                      FARS_hosp_time['DAY_x'].astype(str) + ':'+
                                                      FARS_hosp_time['HOSP_HR'].astype(str) + ':' +
                                                           FARS_hosp_time['HOSP_MN'].astype(str), 
                                                            format='%Y:%m:%d:%H:%M'))




In [ ]:
FARS_hosp_time['Time_to_Accident'] = (FARS_hosp_time['Arrival_Time'] - 
                                        FARS_hosp_time['Notification_Time'])

FARS_hosp_time['Time_to_Hospital'] = (FARS_hosp_time['Hospital_Time'] - 
                                         FARS_hosp_time['Arrival_Time'])

FARS_hosp_time['Total_Time'] = (FARS_hosp_time['Hospital_Time'] - 
                                         FARS_hosp_time['Notification_Time'])
FARS_hosp_time

In [ ]:
#Convert Lat and Lon to radians in the FARS data to extract fatalities in a radius for top ten cities
FARS_16_17_18['Lat_rad'] = FARS_16_17_18['LATITUDE'] * np.pi / 180
FARS_16_17_18['Lon_rad'] = FARS_16_17_18['LONGITUD'] * np.pi / 180

def radius(Lat1, Lon1, Lat2, Lon2):
    # http://www.movable-type.co.uk/scripts/latlong.html?from=48.6093070,-122.4259880&to=48.5928360,-122.4216130
    # http://janmatuschek.de/LatitudeLongitudeBoundingCoordinates
    # Lat/Lon1 is for the top city and Lat/Lon2 is for the accident in FARS data set
    return np.arccos((np.sin(Lat1) * np.sin(Lat2)) + 
                     (np.cos(Lat1) * np.cos(Lat2) * np.cos(Lon2 - (Lon1)))) * 6371

rad = np.sqrt(top_cities['Land'][0])

ny_df = FARS_16_17_18[radius(top_cities['Lat_rad'][0],top_cities['Lon_rad'][0],FARS_16_17_18['Lat_rad'], FARS_16_17_18['Lon_rad']) <= rad].reset_index()

FARS_Top_Cities = pd.DataFrame() 

for i in range(len(top_cities)):
    rad = np.sqrt(top_cities['Land'][i])
    FARS_Top_Cities = FARS_Top_Cities.append(FARS_16_17_18[radius(top_cities['Lat_rad'][i],
                                                                  top_cities['Lon_rad'][i],
                                                                  FARS_16_17_18['Lat_rad'], 
                                                                  FARS_16_17_18['Lon_rad']) <= rad])
    
    
    
FARS_Top_Cities.reset_index(inplace=True)

In [ ]:
# Create Folium Map of Top ten cities and add different attributes to the map such as Bike Stations, Accidents, Hospitals...
m = folium.Map(location=[40, -96], zoom_start=4.4,prefer_canvas=True)

for i in range(len(top_cities)): #Top Ten Cities
    folium.Circle(location=[top_cities['Lat'][i], top_cities['Lon'][i]],
      popup='name',
      radius=15,
      color='blue',
      fill=True,
      fill_color='blue'
   ).add_to(m)
    
for i in range(len(FARS_Top_Cities)): #Crash
    folium.Circle(location=[FARS_Top_Cities['LATITUDE'][i], FARS_Top_Cities['LONGITUD'][i]],
      popup='name',
      radius=80,
      color='red',
      fill=True,
      fill_color='red'
   ).add_to(m)    
    
# for i in range(len(crash_boston_cyclist_df)): #Crash invovled cyclist
#     folium.Circle(location=[crash_boston_cyclist_df['latitude'][i], crash_boston_cyclist_df['longitud'][i]],
#       popup='name',
#       radius=500,
#       color='yellow',
#       fill=True,
#       fill_color='yellow'
#    ).add_to(m)   
    
# for i in range(len(hospitals_df)): #Hospital Location
#     folium.Circle(location=[hospitals_df['LATITUDE'][i], hospitals_df['LONGITUDE'][i]],
#       popup='name',
#       radius=50,
#       color='green',
#       fill=True,
#       fill_color='green'
#    ).add_to(m) 

m